In [14]:
from src.mobj_types.mobj_type import MobjType
from src.mobj_types.value_structure import ValueStructure
from src.mobjs.variable import Variable
from src.scopes.inference_rule import InferenceRule
from src.mobjs.mobj import Mobj
from src.scopes.scope import Scope
from src.scopes.scope_tree import ScopeTree

In [15]:


IsProposition = MobjType("IsProposition", ValueStructure.non_defining(1))
IsSet = MobjType("IsSet", ValueStructure.non_defining(1))

Implication = MobjType("Implication", ValueStructure.non_defining(2))
Negation = MobjType("Negation", ValueStructure.non_defining(1))
Membership = MobjType("Membership", ValueStructure.non_defining(2))

Forall = MobjType("Forall", ValueStructure.defines_first(2))
Exists = MobjType("Exists", ValueStructure.defines_first(2))
ExistsOne = MobjType("ExistsOne", ValueStructure.defines_first(2))

In [ ]:
class I(InferenceRule):
    def __init__(self):
        pass
        
    def types_are_valid(self) -> bool:
        return True
    
    def is_applicable(self, scope: Scope) -> bool:
        return True
    
    def infer(self, scope : Scope):
        pass

In [16]:
class AddAxiom(InferenceRule):
    def __init__(self, index : int):
        self.index = index
        
    def types_are_valid(self) -> bool:
        return True
    
    def infer(self, scope: Scope, *args: Mobj) -> bool:
        axiom = scope.tree.get_axiom(self.index)
        if axiom is None:
            return False
        
        # Add axiom variables to scope
        old_variables = scope._scope_quantifier_variables.copy()
        
        scope._scope_quantifier_variables.update(axiom.variables)
        if scope.add_true_statement(axiom)[0]:
            return True
        
        scope._scope_quantifier_variables = old_variables
        return False

In [27]:
class AddParentProposition(InferenceRule):
    def __init__(self, number_of_generations : int, index : int):
        self.number_of_generations = number_of_generations
        self.index = index
        
    def types_are_valid(self) -> bool:
        return True

    
    def infer(self, scope : Scope) -> bool:
        # We assume that the father scope is not None and that the index is valid
        father_scope = scope.get_father_scope(self.number_of_generations)
        
        if not father_scope:
            return False
        
        proposition = father_scope.get_true_statement(self.index)
        if not proposition:
            return False
        
        return scope.add_true_statement(proposition)[0]
        

In [17]:
tree = ScopeTree(
    [IsProposition, Forall],
    {"AddAxiom" : AddAxiom},
)
tree.add_axiom(Forall(Variable("A"), Variable("A")))